# Neurorobotics with PyNN and PyBullet

In this Notebook we will build and execute a neuro-robotics experiment. Für the neural simulation we will use [PyNN](https://neuralensemble.org/PyNN/) and [Nest](https://www.nest-initiative.org/?page=Software). For the simulation of the physic we use [PyBullet](https://github.com/bulletphysics/bullet3/tree/master/examples/pybullet). 

This experiment is a re-creation of the [braitenberg-husky-experiment](https://bitbucket.org/hbpneurorobotics/experiments/src/development/braitenberg_husky/) in the neurorobotics plattform of the human brain project. 

We start by importing all modules we will use in this notebook. Make sure to install only python**3**-versions of all dependencies. 

You can not just install these via pip:

- _pybullet_ needs to be installed from source to get numpy-support. Otherwise it will be slower, and getCameraImage doesn't return a numpy-array. 
- _nest_ can only be installed from source. We must use version 2.16, because the current master is not yet compatible with _pynn_. Also _nest_ is compiled with _libnreuosim_, which needs a workaround until [the PR](https://github.com/nest/nest-simulator/pull/1235) is merged. I'm not 100% certain if _libneurosim_ is even required for this project, but _nest_ gives a warning if it's missing, so we will install it. 
- there is a warning "UserWarning: Unable to install NEST extensions. Certain models may not be available", which doesn't seem to affect this project. Please ignore it. 

In [1]:
import time
import numpy as np
import csv
import random

import subprocess
import re

from matplotlib import pyplot as plt
from matplotlib import animation
from pyNN.utility.plotting import Figure, Panel
from quantities import mV

In [2]:
def isFinished(processes):
    
    for process in processes:
        if(process.poll() == None):
            return False
    
    return True

In [3]:
output_min = []
error_min = 200
#number_params = 4
number_params = 8

print("Starting")

for N in range(200):
    
    # Start time measurement
    start = time.time()
    
    weights_list = []
    processes = []
    
    # Execute all subprocesses and save outputs into files
    for i in range(40):
        
        # Generate Random weights
        weights = []
        for _ in range(number_params):
            #weights.append( str(10**random.uniform(-4, -2)) ) 
            weights.append( str(10**random.uniform(-5, 3)) ) 

        # Spawn the subprocesses and write outputs into files
        filename = "temp/log" + str(i) + ".txt"
        with open(filename, "w") as file:
            command = ['python', 'OptimizationJob_BigBrain.py']
            command.extend(weights)
            processes.append( subprocess.Popen(command, stdout=file) ) 
            
        # Append current weights to weights list
        weights_list.append(weights)
            
    # Wait until all processes are finished
    while isFinished(processes) == False:
        time.sleep(0.5)
    
    # Read outputs
    for i in range(40):

        with open("temp/log" + str(i) + ".txt") as file:
            filetext = file.read()

            # Find fitness value through regex
            matches = re.findall("(.*[^=])=(.*)", filetext)
            error = float(matches[0][1])

            # A new minimum is found
            if error < error_min:
                error_min = error
                print(weights_list[i])
                print(error)

    # Time measurement
    end = time.time()
    
    # Ausgabe
    print('Time taken in seconds -', end - start)
    
    
print("Finished")

Starting
['570.0308809139513', '0.09669200296988162', '0.9599832270799492', '3.3438245162895716e-05', '120.83823313318939', '3.6779739897150896', '0.20049371382800812', '7.44233858417996e-05']
1.8505161483806634
['170.09529093594887', '152.8743977577786', '0.0006320510995423669', '42.33463099103715', '11.517769418946848', '1.6165551955831918', '0.3153836221543736', '0.012213925256479028']
1.8453810984922319
['13.443274306464476', '0.13714375998793357', '0.029768886325137446', '0.005078858935536639', '0.0001948876051222712', '4.926939518264329', '260.48666952129327', '0.6700206412814903']
1.657026811840161
['3.174993271153038', '115.34890045061384', '18.18347969209679', '4.615220558588311e-05', '0.38905235777796093', '0.39080525966544943', '242.48995020655363', '0.0002016579780658747']
1.6061361730714598
Time taken in seconds - 252.9256100654602
['0.0013781945124330395', '0.000486347209636169', '402.8345945035941', '0.038407771280169554', '359.51441534964647', '0.43360699441432515', '22

KeyboardInterrupt: 

### Time measurement for small brain

| Number of subprocesses   |  Elapsed Time   |  Elapsed Time per subprocess  |
|--------------------------|-----------------|-------------------------------|
| 1                        |    39.06 s      |         39.06 s               |
| 5                        |    42.63 s      |          8.52 s               |
| 10                       |    44.72 s      |          4.47 s               |
| 20                       |    49.58 s      |          2.48 s               |
| 30                       |    66.34 s      |          2.21 s               |
| 40                       |    73.10 s      |          1.83 s               |
| 50                       |    92.97 s      |          1.86 s               |
| 80                       |   149.80 s      |          1.87 s               |
|100                       |   188.15 s      |          1.88 s               |
|200                       |   382.65 s      |          1.91 s               |
|300                       |   592.33 s      |          1.97 s               |
|500                       |   993.66 s      |          1.99 s               |
|800                       |  1599.73 s      |          1.99 s               |